In [0]:
#from pyspark.sql import SparkSession

#spark=SparkSession.builder.appName('transformation').getOrCreate()



In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 spark://10.94.225.201:7077 
 AppName 
 Databricks Shell

In [0]:
%fs
ls 

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/mnt/,mnt/,0
dbfs:/tmp/,tmp/,0
dbfs:/user/,user/,0


In [0]:
%fs
ls "/mnt"

path,name,size
dbfs:/mnt/databricks-yiding/,databricks-yiding/,0
dbfs:/mnt/imba-yiding/,imba-yiding/,0


In [0]:
'''
#read IAM user kay value pair for connecting aws account

file_location="dbfs:/FileStore/tables/aws_keys/aws_key.csv"
file_type="csv"
first_row_header="true"
delimiter=','

AWS_IAM_df=spark.read.format(file_type) \
  .option("header", first_row_header) \
  .option("sep", delimiter) \
  .load(file_location)




#collect key value pair strings
IAM_ID=AWS_IAM_df.select("IAM_ID").collect()[0].IAM_ID
IAM_SECRET=AWS_IAM_df.select('IAM_SECRET').collect()[0].IAM_SECRET
#print(IAM_ID)
#print(IAM_SECRET)

#encode secret
IAM_SECRET_encode=urllib.parse.quote(IAM_SECRET,"")
#print(IAM_SECRET_encode)




  
aws_s3_bucket="databricks-yiding"
mount_name="/mnt/databricks-yiding"
s3_location="s3n://{0}:{1}@{2}".format(IAM_ID,IAM_SECRET_encode,aws_s3_bucket)
dbutils.fs.mount(s3_location,mount_name)
#dbutils.fs.rm('dbfs:/mnt/databricks-yiding/',True)


'''

Out[5]: '\n#read IAM user kay value pair for connecting aws account\n\nfile_location="dbfs:/FileStore/tables/aws_keys/aws_key.csv"\nfile_type="csv"\nfirst_row_header="true"\ndelimiter=\',\'\n\nAWS_IAM_df=spark.read.format(file_type) .option("header", first_row_header) .option("sep", delimiter) .load(file_location)\n \n \naws_s3_bucket="databricks-yiding"\nmount_name="/mnt/databricks-yiding"\ns3_location="s3n://{0}:{1}@{2}".format(IAM_ID,IAM_SECRET_encode,aws_s3_bucket)\ndbutils.fs.mount(s3_location,mount_name)\n#dbutils.fs.rm(\'dbfs:/mnt/databricks-yiding/\',True)\n\n\n'

In [0]:

#mount s3 buckt on dbfs
from pyspark.sql.functions import *
import urllib

file_location="dbfs:/FileStore/tables/aws_keys/aws_key.csv"
file_type="csv"
first_row_header="true"
delimiter=','

AWS_IAM_df=spark.read.format(file_type) \
  .option("header", first_row_header) \
  .option("sep", delimiter) \
  .load(file_location)


#collect key value pair strings
IAM_ID=AWS_IAM_df.select("IAM_ID").collect()[0].IAM_ID
IAM_SECRET=AWS_IAM_df.select('IAM_SECRET').collect()[0].IAM_SECRET
#print(IAM_ID)
#print(IAM_SECRET)

#encode secret
IAM_SECRET_encode=urllib.parse.quote(IAM_SECRET,"")
#print(IAM_SECRET_encode)


aws_s3_bucket='imba-yiding'
mount_name='/mnt/imba-yiding'
s3_location='s3n://{0}:{1}@{2}'.format(IAM_ID,IAM_SECRET_encode,aws_s3_bucket)
dbutils.fs.mount(s3_location,mount_name)

Out[7]: True

In [0]:
%fs
ls "/mnt"

path,name,size
dbfs:/mnt/databricks-yiding/,databricks-yiding/,0
dbfs:/mnt/imba-yiding/,imba-yiding/,0


In [0]:
%fs
ls '/mnt/imba-yiding/data/'

path,name,size
dbfs:/mnt/imba-yiding/data/aisles/,aisles/,0
dbfs:/mnt/imba-yiding/data/departments/,departments/,0
dbfs:/mnt/imba-yiding/data/order_products/,order_products/,0
dbfs:/mnt/imba-yiding/data/orders/,orders/,0
dbfs:/mnt/imba-yiding/data/products/,products/,0


In [0]:
#read data from mounted s3 into dataframe
order_df=spark.read.option('header','true').csv('/mnt/imba-yiding/data/orders/',inferSchema=True)
product_df=spark.read.option('header','true').csv('/mnt/imba-yiding/data/products/',inferSchema=True)
order_product_df=spark.read.option('header','true').csv('/mnt/imba-yiding/data/order_products/',inferSchema=True)
department_df=spark.read.option('header','true').csv('/mnt/imba-yiding/data/departments/',inferSchema=True)
aisles_df=spark.read.option('header','true').csv('/mnt/imba-yiding/data/aisles/',inferSchema=True)

#PREVIEW DATA

In [0]:
order_df.show()

+--------+-------+--------+------------+---------+-----------------+----------------------+
order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
 2539329| 1| prior| 1| 2| 8| null|
 2398795| 1| prior| 2| 3| 7| 15.0|
 473747| 1| prior| 3| 3| 12| 21.0|
 2254736| 1| prior| 4| 4| 7| 29.0|
 431534| 1| prior| 5| 4| 15| 28.0|
 3367565| 1| prior| 6| 2| 7| 19.0|
 550135| 1| prior| 7| 1| 9| 20.0|
 3108588| 1| prior| 8| 1| 14| 14.0|
 2295261| 1| prior| 9| 1| 16| 0.0|
 2550362| 1| prior| 10| 4| 8| 30.0|
 1187899| 1| train| 11| 4| 8| 14.0|
 2168274| 2| prior| 1| 2| 11| null|
 1501582| 2| prior| 2| 5| 10| 10.0|
 1901567| 2| prior| 3| 1| 10| 3.0|
 738281| 2| prior| 4| 2| 10| 8.0|
 1673511| 2| prior| 5| 3| 11| 8.0|
 1199898| 2| prior| 6| 2| 9| 13.0|
 3194192| 2| prior| 7| 2| 12| 14.0|
 788338| 2| prior| 8| 1| 15| 27.0|
 1718559| 2| prior| 9| 2| 9| 8.0|
+--------+-------+--------+------------+---------+-----------------+----------------------+
only showing top 20 rows

In [0]:
order_df.printSchema()

root
-- order_id: integer (nullable = true)
-- user_id: integer (nullable = true)
-- eval_set: string (nullable = true)
-- order_number: integer (nullable = true)
-- order_dow: integer (nullable = true)
-- order_hour_of_day: integer (nullable = true)
-- days_since_prior_order: double (nullable = true)

In [0]:
order_df.columns

Out[6]: ['order_id',
 'user_id',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order']

In [0]:
order_product_df.show()

+--------+----------+-----------------+---------+
order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
 2| 33120| 1| 1|
 2| 28985| 2| 1|
 2| 9327| 3| 0|
 2| 45918| 4| 1|
 2| 30035| 5| 0|
 2| 17794| 6| 1|
 2| 40141| 7| 1|
 2| 1819| 8| 1|
 2| 43668| 9| 0|
 3| 33754| 1| 1|
 3| 24838| 2| 1|
 3| 17704| 3| 1|
 3| 21903| 4| 1|
 3| 17668| 5| 1|
 3| 46667| 6| 1|
 3| 17461| 7| 1|
 3| 32665| 8| 1|
 4| 46842| 1| 0|
 4| 26434| 2| 1|
 4| 39758| 3| 1|
+--------+----------+-----------------+---------+
only showing top 20 rows

In [0]:
order_product_df.columns

Out[20]: ['order_id', 'product_id', 'add_to_cart_order', 'reordered']

In [0]:
aisles_df.show(5)

+--------+--------------------+
aisle_id| aisle|
+--------+--------------------+
 1|prepared soups sa...|
 2| specialty cheeses|
 3| energy granola bars|
 4| instant foods|
 5|marinades meat pr...|
+--------+--------------------+
only showing top 5 rows

In [0]:
product_df.show(5)

+----------+--------------------+--------+-------------+
product_id| product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
 1|Chocolate Sandwic...| 61| 19|
 2| All-Seasons Salt| 104| 13|
 3|Robust Golden Uns...| 94| 7|
 4|Smart Ones Classi...| 38| 1|
 5|Green Chile Anyti...| 5| 13|
+----------+--------------------+--------+-------------+
only showing top 5 rows

In [0]:
department_df.show(5)

+-------------+----------+
department_id|department|
+-------------+----------+
 1| frozen|
 2| other|
 3| bakery|
 4| produce|
 5| alcohol|
+-------------+----------+
only showing top 5 rows

In [0]:
#create temp view to do sql query
order_df.createOrReplaceTempView('order_table')
order_product_df.createOrReplaceTempView('order_product_table')
department_df.createOrReplaceTempView('department_table')
product_df.createOrReplaceTempView('product_table')
aisles_df.createOrReplaceTempView('aisles_table')

#Transformation

In [0]:
%sql
SELECT ot.*, pt.product_id, pt.add_to_cart_order, pt.reordered
FROM order_table ot
JOIN order_product_table pt
ON ot.order_id = pt.order_id
WHERE ot.eval_set = 'prior'

order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
31,201744,prior,7,6,15,14.0,48946,1,1
31,201744,prior,7,6,15,14.0,22935,2,1
31,201744,prior,7,6,15,14.0,21903,3,1
31,201744,prior,7,6,15,14.0,31717,4,1
31,201744,prior,7,6,15,14.0,4920,5,1
31,201744,prior,7,6,15,14.0,25647,6,1
31,201744,prior,7,6,15,14.0,49156,7,1
31,201744,prior,7,6,15,14.0,29926,8,1
31,201744,prior,7,6,15,14.0,47900,9,1
31,201744,prior,7,6,15,14.0,9436,10,0


In [0]:
#collect necessary data together
order_products_prior=spark.sql("""
SELECT ot.*, pt.product_id, pt.add_to_cart_order, pt.reordered
FROM order_table ot
JOIN order_product_table pt
ON ot.order_id = pt.order_id
WHERE ot.eval_set = 'prior'
""")
order_products_prior.show()

+--------+-------+--------+------------+---------+-----------------+----------------------+----------+-----------------+---------+
order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|product_id|add_to_cart_order|reordered|
+--------+-------+--------+------------+---------+-----------------+----------------------+----------+-----------------+---------+
 31| 201744| prior| 7| 6| 15| 14.0| 48946| 1| 1|
 31| 201744| prior| 7| 6| 15| 14.0| 22935| 2| 1|
 31| 201744| prior| 7| 6| 15| 14.0| 21903| 3| 1|
 31| 201744| prior| 7| 6| 15| 14.0| 31717| 4| 1|
 31| 201744| prior| 7| 6| 15| 14.0| 4920| 5| 1|
 31| 201744| prior| 7| 6| 15| 14.0| 25647| 6| 1|
 31| 201744| prior| 7| 6| 15| 14.0| 49156| 7| 1|
 31| 201744| prior| 7| 6| 15| 14.0| 29926| 8| 1|
 31| 201744| prior| 7| 6| 15| 14.0| 47900| 9| 1|
 31| 201744| prior| 7| 6| 15| 14.0| 9436| 10| 0|
 53| 87889| prior| 5| 4| 20| 3.0| 24852| 1| 1|
 53| 87889| prior| 5| 4| 20| 3.0| 37646| 2| 1|
 53| 87889| prior| 5| 4| 20| 3.0| 47209| 3| 1|
 53| 87889| prior| 5| 4| 20| 3.0| 36606| 4| 0|
 53| 87889| prior| 5| 4| 20| 3.0| 32442| 5| 1|
 53| 87889| prior| 5| 4| 20| 3.0| 14467| 6| 1|
 53| 87889| prior| 5| 4| 20| 3.0| 40706| 7| 1|
 53| 87889| prior| 5| 4| 20| 3.0| 5876| 8| 1|
 53| 87889| prior| 5| 4| 20| 3.0| 22298| 9| 1|
 53| 87889| prior| 5| 4| 20| 3.0| 38827| 10| 1|
+--------+-------+--------+------------+---------+-----------------+----------------------+----------+-----------------+---------+
only showing top 20 rows

In [0]:
user_features_1=spark.sql("""
SELECT user_id,
Max(order_number) AS user_orders,
Sum(days_since_prior_order) AS user_period,
Avg(days_since_prior_order) AS user_mean_days_since_prior
FROM order_table 
GROUP BY user_id 
""")

In [0]:
#calculate the total number of products, total number of distinct products, and user reorder ratio
#(number of reordered = 1 divided by number of order_number > 1)

order_products_prior.createOrReplaceTempView('order_products_prior_table')

user_features_2=spark.sql("""
SELECT user_id,Count(*) AS user_total_products, Count(DISTINCT product_id) AS user_distinct_products,
Sum(CASE WHEN reordered = 1 THEN 1 ELSE 0 END) / Cast(Sum(CASE WHEN order_number > 1 THEN 1 ELSE 0 END) AS DOUBLE) AS user_reorder_ratio
FROM order_products_prior_table 
GROUP BY user_id
""")

In [0]:
#for each user and product, calculate the total number of orders, minimum order_number, maximum order_number and average add_to_cart_order.

up_features=spark.sql("""
select user_id, product_id, count(*) AS up_orders, Min(order_number) AS up_first_order, 
Max(order_number) as up_last_order, Avg(add_to_cart_order) As up_average_cart_position
from order_products_prior_table
GROUP BY user_id,product_id
""")

In [0]:
#calculate the sequence of product purchase for each user
#for each product, calculate the count, sum of reordered, count of product_seq_time = 1 and count of product_seq_time = 2.

prd_features=spark.sql("""
SELECT product_id,Count(*) AS prod_orders,Sum(reordered) AS prod_reorders,
Sum(CASE WHEN product_seq_time = 1 THEN 1 ELSE 0 END) AS prod_first_orders, Sum(CASE WHEN product_seq_time = 2 THEN 1 ELSE 0 END) AS prod_second_orders
FROM (
    SELECT *, Rank()OVER (partition BY user_id, product_id ORDER BY order_number) AS product_seq_time
    FROM order_products_prior_table
    )
GROUP BY product_id
""")

In [0]:
#persist some RDD dataframe for mutiple use

from pyspark import StorageLevel
prd_features.persist(StorageLevel.MEMORY_AND_DISK)
user_features_1.persist(StorageLevel.MEMORY_AND_DISK)
user_features_2.persist(StorageLevel.MEMORY_AND_DISK)
up_features.persist(StorageLevel.MEMORY_AND_DISK)

Out[19]: DataFrame[user_id: int, product_id: int, up_orders: bigint, up_first_order: int, up_last_order: int, up_average_cart_position: double]

#join all dataframes together to get the final transformed data

In [0]:
user_features_1.show(5)
user_features_2.show(5)

+-------+-----------+-----------+--------------------------+
user_id|user_orders|user_period|user_mean_days_since_prior|
+-------+-----------+-----------+--------------------------+
 18654| 16| 117.0| 7.8|
 18800| 10| 180.0| 20.0|
 18866| 39| 253.0| 6.657894736842105|
 18911| 15| 105.0| 7.5|
 18944| 5| 102.0| 25.5|
+-------+-----------+-----------+--------------------------+
only showing top 5 rows

+-------+-------------------+----------------------+------------------+
user_id|user_total_products|user_distinct_products|user_reorder_ratio|
+-------+-------------------+----------------------+------------------+
 7554| 298| 124|0.6062717770034843|
 103747| 409| 72|0.8575063613231552|
 8086| 489| 73|0.8524590163934426|
 20924| 643| 266|0.5993640699523053|
 146224| 1050| 204|0.8237585199610516|
+-------+-------------------+----------------------+------------------+
only showing top 5 rows

In [0]:
#join user1 and user2 with key user_id, inner join

users = user_features_1.join(user_features_2,'user_id', 'inner')
users.show(5)

+-------+-----------+-----------+--------------------------+-------------------+----------------------+------------------+
user_id|user_orders|user_period|user_mean_days_since_prior|user_total_products|user_distinct_products|user_reorder_ratio|
+-------+-----------+-----------+--------------------------+-------------------+----------------------+------------------+
 7554| 17| 141.0| 8.8125| 298| 124|0.6062717770034843|
 103747| 28| 214.0| 7.925925925925926| 409| 72|0.8575063613231552|
 8086| 100| 350.0| 3.5353535353535355| 489| 73|0.8524590163934426|
 20924| 65| 360.0| 5.625| 643| 266|0.5993640699523053|
 146224| 55| 362.0| 6.703703703703703| 1050| 204|0.8237585199610516|
+-------+-----------+-----------+--------------------------+-------------------+----------------------+------------------+
only showing top 5 rows

In [0]:
#join up_features prd_features and users together

final_df=(up_features.join(users,'user_id','inner')).join(prd_features,'product_id','inner')
final_df.show(5)

+----------+-------+---------+--------------+-------------+------------------------+-----------+-----------+--------------------------+-------------------+----------------------+------------------+-----------+-------------+-----------------+------------------+
product_id|user_id|up_orders|up_first_order|up_last_order|up_average_cart_position|user_orders|user_period|user_mean_days_since_prior|user_total_products|user_distinct_products|user_reorder_ratio|prod_orders|prod_reorders|prod_first_orders|prod_second_orders|
+----------+-------+---------+--------------+-------------+------------------------+-----------+-----------+--------------------------+-------------------+----------------------+------------------+-----------+-------------+-----------------+------------------+
 9366| 73933| 1| 7| 7| 17.0| 50| 361.0| 7.36734693877551| 477| 99| 0.816414686825054| 6763| 3329| 3434| 1298|
 19678| 69637| 6| 1| 10| 21.5| 15| 194.0| 13.857142857142858| 352| 108|0.8413793103448276| 32534| 19933| 12601| 5938|
 5479| 29054| 17| 2| 27| 1.9411764705882353| 42| 258.0| 6.2926829268292686| 173| 65|0.6467065868263473| 30937| 25173| 5764| 3658|
 37164| 83693| 1| 3| 3| 15.0| 5| 88.0| 22.0| 65| 58| 0.14| 331| 215| 116| 55|
 4656| 6336| 1| 1| 1| 2.0| 19| 226.0| 12.555555555555555| 191| 91|0.5405405405405406| 9989| 4911| 5078| 1777|
+----------+-------+---------+--------------+-------------+------------------------+-----------+-----------+--------------------------+-------------------+----------------------+------------------+-----------+-------------+-----------------+------------------+
only showing top 5 rows

In [0]:
final_df.count()

Out[30]: 13307953

#put final data to data lake curated stage

In [0]:
final_df.repartition(1).write.format('csv') \
.option('header','true') \
.save('/mnt/imba-yiding/curated/version1')

In [0]:
# release memory
prd_features.unpersist()
user_features_1.unpersist()
user_features_2.unpersist()
up_features.unpersist()


Out[33]: DataFrame[user_id: int, product_id: int, up_orders: bigint, up_first_order: int, up_last_order: int, up_average_cart_position: double]